In [26]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np

In [22]:
from lib.BBData import character_dict
from lib.BBDataLoad import open_char_dataset, process_char_dataset

In [16]:
characters = list(character_dict.keys())
characters_noDefault = list(character_dict.keys())
characters_noDefault.remove('Default')

character = 'Barney'
base_folder = os.getcwd()
data_folder = os.path.join(base_folder, 'Data', 'Characters')
model_folder = os.path.join(base_folder, 'models')
embedder_path = os.path.join(os.getcwd(), 'models', 'distilbert_embedder')
bertopic_path = os.path.join(os.getcwd(), 'models', 'bertopic_classifier', 'n_sentences7')
out_folder = os.getcwd().join('tmp')
figsize = (8, 8)

In [12]:
def load_himym_friends_tbbt_dataset(sources_folder):
    dataframe_rows = []
    # Get number of documents and their names
    documents_n = len(os.listdir(sources_folder))
    documents_names = os.listdir(sources_folder)
    # Loop over documents
    for i in tqdm(range(documents_n)):
        # Extract filename which correspond to the link of the episode
        filename = documents_names[i]
        # the last 5 chars takes the form `sxe` with s the number of the current serie and
        # and e as the number of the episode
        sources_label = filename[:-4]
        # Open document
        with open(os.path.join(sources_folder, filename), encoding="utf8") as file:
            # Loop over lines (= words)
            for line in file.readlines():
                dataframe_row = {
                    "source": sources_label,
                    "line": line,
                }
                dataframe_rows.append(dataframe_row)
    # Build the dataframe from the words
    df = pd.DataFrame(dataframe_rows)
    return df

In [20]:
df = open_char_dataset(character, base_folder)
df.head()

100%|██████████| 208/208 [00:03<00:00, 64.51it/s]


,source,line
0,01x01,Pilot\n
1,01x01,Scene One\n
2,01x01,[Title: The Year 2030]\n
3,01x01,"Narrator: Kids, I'm going to tell you an incre..."
4,01x01,Son: Are we being punished for something?\n


In [23]:
df_prepr = process_char_dataset(df, character)
df_prepr

/app/notebooks/lib/BBDataLoad.py:167: FutureWarning: The default value of regex will change from True to False in a future version.
  df['line'] = df['line'].str.replace(r"\(.*\)", "")
/app/notebooks/lib/BBDataLoad.py:169: FutureWarning: The default value of regex will change from True to False in a future version.
  df['line'] = df['line'].str.replace(r"[\/(){}\[\]\|@_#]|\\t|\\n", " ")
/app/notebooks/lib/BBDataLoad.py:171: FutureWarning: The default value of regex will change from True to False in a future version.
  df['line'] = df['line'].str.replace(r"[^.\',;:?!0-9a-zA-Z \-]", "")
/app/notebooks/lib/BBDataLoad.py:173: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['character', 'line']] = df['line'].str.split(":", 1, expand=True)


,source,line,character
0,01x01,"Kids, I'm going to tell you an incredible stor...",Narrator
1,01x01,Are we being punished for something?,Son
2,01x01,No,Narrator
3,01x01,"Yeah, is this going to take a while?",Daughter
4,01x01,"Yes. Twenty-five years ago, before I was dad,...",Narrator
...,...,...,...
31771,09x24,Aunt Lily wasn't wrong.,NARRATOR
31772,09x24,"00 a.m. Christmas morning, every sleepy Sunday...",I carried that lesson with me through every st...
31773,09x24,T.M.,Excuse me. It even has my initials on it right...
31774,09x24,T.M.,Those are my initials


In [37]:
until_id = df_prepr[df_prepr['source']=='07x01'].index[0]
df_prepr = df_prepr[:until_id]
df_prepr

,source,line,character
0,01x01,"Kids, I'm going to tell you an incredible stor...",Narrator
1,01x01,Are we being punished for something?,Son
2,01x01,No,Narrator
3,01x01,"Yeah, is this going to take a while?",Daughter
4,01x01,"Yes. Twenty-five years ago, before I was dad,...",Narrator
...,...,...,...
29743,06x24,"It's never too late, Barney.",Norah
29744,06x24,Challenge accepted.,Barney
29745,06x24,Best man? You're being summoned.,Lily
29746,06x24,"Geez, what now? I heard the groom needed me.",Ted


In [38]:
characters_HIMYM = df_prepr['character'].to_list()
len(df_prepr['character'].unique())

813

In [39]:
df_prepr['character'].unique()

array(['Narrator', 'Son', 'Daughter', 'Marshall', 'Ted', 'Barney',
       'Yasmine', 'Lily', 'Robin', 'Cabdriver', "Robin's Dumped Friend",
       'Producer', 'Waitor', 'Ranjit', 'Lily, Marshall and Barney',
       'Son and Daughter', 'Rangit', 'Marshal', 'Carl', 'Cameraman',
       'Leroy', 'Lily and Marshall', 'Fantasy Girl', 'Tatiana',
       'Lily and Ted', 'Crowd', 'Carlos', 'Barney and Ted',
       'Marshall, Lily and Ted', 'Mashall, Lily and Ted', 'Guy 1',
       'Laura', 'Fight Attendant', 'Guy 2', 'Guy 3', 'Officer McNeil',
       'bmb Squad Guy', 'Derrick', 'Dana', 'Sascha', 'Cabdriver 2',
       'Cute Girl', 'Stefanie', 'Marshall and Ted', 'Mr. Adams',
       'Natalie', 'One Guest', 'All', 'Henry', 'Waiter', 'Claire',
       'Bradley', 'Chris', 'Austin', 'Kelly', 'Bartender', 'Phil',
       'Man on Street', 'Doorman 2', 'Woman', 'Coat Check Girl',
       'Barney, Ted and Robin', 'Future Ted', 'Lily ', 'Barney ',
       'Marshall, Lily, Barney', 'Lily, Marshall, Barney', 'Mik